In [33]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity


In [34]:
# Load datasets
ireviews_df = pd.read_csv('data/Books_rating.csv')
ibooks_df = pd.read_csv('data/books_data.csv')

ireviews_df

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t..."
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...
...,...,...,...,...,...,...,...,...,...,...
2999995,B000NSLVCU,The Idea of History,NaN,NaN,NaN,14/19,4.0,937612800,Difficult,"This is an extremely difficult book to digest,..."
2999996,B000NSLVCU,The Idea of History,NaN,A1SMUB9ASL5L9Y,jafrank,1/1,4.0,1331683200,Quite good and ahead of its time occasionally,This is pretty interesting. Collingwood seems ...
2999997,B000NSLVCU,The Idea of History,NaN,A2AQMEKZKK5EE4,"L. L. Poulos ""Muslim Mom""",0/0,4.0,1180224000,Easier reads of those not well versed in histo...,"This is a good book but very esoteric. ""What i..."
2999998,B000NSLVCU,The Idea of History,NaN,A18SQGYBKS852K,"Julia A. Klein ""knitting rat""",1/11,5.0,1163030400,"Yes, it is cheaper than the University Bookstore","My daughter, a freshman at Indiana University,..."


In [35]:
ibooks_df.dropna(subset=['ratingsCount'], inplace=True)
ibooks_df.head()


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
5,The Church of Christ: A Biblical Ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,['Everett Ferguson'],http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0
7,A Walk in the Woods: a Play in Two Acts,NaN,['Lee Blessing'],NaN,http://books.google.nl/books?id=6HDOwAEACAAJ&d...,NaN,1988,http://books.google.nl/books?id=6HDOwAEACAAJ&d...,NaN,3.0
31,Voices from the Farm: Adventures in Community ...,"Twenty-five years ago, at the height of the co...",['Rupert Fike'],http://books.google.com/books/content?id=IjTAB...,http://books.google.nl/books?id=IjTABgAAQBAJ&p...,Book Publishing Company,2012-08-21,https://play.google.com/store/books/details?id...,['Biography & Autobiography'],1.0
33,The Battleship Bismarck,The Bismarck is perhaps the most famous – and ...,['Stefan Draminski'],http://books.google.com/books/content?id=nxttD...,http://books.google.nl/books?id=nxttDwAAQBAJ&p...,Bloomsbury Publishing,2018-09-20,https://play.google.com/store/books/details?id...,['History'],1.0
35,Mini-mysteries,NaN,NaN,http://books.google.com/books/content?id=mhqyg...,http://books.google.nl/books?id=mhqygn6STHgC&p...,Remedia Publications,NaN,http://books.google.nl/books?id=mhqygn6STHgC&d...,NaN,1.0


In [36]:
reviews_df = ireviews_df
books_df = ibooks_df

In [37]:
# drop non-relevant columns
books_df = books_df.loc[:, ['Title', 'categories', 'authors', 'ratingsCount']]
reviews_df.drop(['Price', 'profileName', 'review/time', 'Id', 'review/helpfulness', 'review/summary'], axis=1, inplace=True)

In [38]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder to the categories and transform them to numerical values
books_df['category_encoded'] = label_encoder.fit_transform(books_df['categories'])
books_df['author_encoded'] = label_encoder.fit_transform(books_df['authors'])
books_df.drop(['authors', 'categories'], axis=1, inplace=True)

In [39]:
reviews_df.head()

,Title,User_id,review/score,review/text
0,Its Only Art If Its Well Hung!,AVCGYZL8FQQTD,4.0,This is only for Julie Strain fans. It's a col...
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0,I don't care much for Dr. Seuss but after read...
2,Dr. Seuss: American Icon,A3UH4UZ4RSVO82,5.0,"If people become the books they read and if ""t..."
3,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0,Philip Nel - Dr. Seuss: American IconThis is b...


In [40]:
books_df.head()

,Title,ratingsCount,category_encoded,author_encoded
5,The Church of Christ: A Biblical Ecclesiology ...,5.0,2318,9199
7,A Walk in the Woods: a Play in Two Acts,3.0,2598,17915
31,Voices from the Farm: Adventures in Community ...,1.0,549,26384
33,The Battleship Bismarck,1.0,1687,27762
35,Mini-mysteries,1.0,2598,31419


In [45]:
# Count the number of reviews per user
review_counts = reviews_df['User_id'].value_counts()

# Filter the Series to keep only users with 7 or more reviews
users_with_7_plus_reviews = review_counts[review_counts >= 7]

# Get the number of users who made 7 or more reviews
num_users_with_7_plus_reviews = len(users_with_3_plus_reviews)

print(f"Number of users who made 7 or more reviews: {num_users_with_7_plus_reviews}")

Number of users who made 7 or more reviews: 52799


In [46]:
# Alter reviews_df to only retain users who made 6+ reviews
# Count the number of reviews per user
review_counts = reviews_df['User_id'].value_counts()

# Filter the Series to get user IDs with 5 or more reviews
user_ids_with_6_plus_reviews = review_counts[review_counts >= 7].index

# Filter the DataFrame to keep only rows with user IDs in user_ids_with_6_plus_reviews
filtered_df = reviews_df[reviews_df['User_id'].isin(user_ids_with_6_plus_reviews)]

print(f"Original DataFrame shape: {reviews_df.shape}")
print(f"Filtered DataFrame shape: {filtered_df.shape}")

Original DataFrame shape: (3000000, 4)
Filtered DataFrame shape: (997771, 4)


In [47]:
books_df.dropna(inplace=True)
filtered_df = filtered_df.dropna().copy(deep=True)  # Assign the result back to filtered_df
filtered_df.shape



(997707, 4)

In [48]:
filtered_df.head()

,Title,User_id,review/score,review/text
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0,I don't care much for Dr. Seuss but after read...
3,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0,"Theodore Seuss Geisel (1904-1991), aka &quot;D..."
4,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0,Philip Nel - Dr. Seuss: American IconThis is b...
5,Dr. Seuss: American Icon,A2F6NONFUDB6UK,4.0,"""Dr. Seuss: American Icon"" by Philip Nel is a ..."
6,Dr. Seuss: American Icon,A14OJS0VWMOSWO,5.0,Theodor Seuss Giesel was best known as 'Dr. Se...


In [49]:
from tqdm import tqdm
# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Define your compute_sentiment function with debugging
def compute_sentiment(text):
    try:
        score = sia.polarity_scores(text)['compound']
        return score
    except Exception as e:
        print(f"Error processing text: {text[:30]} - {e}")
        return None

# Determine batch size
batch_size = 10000
num_batches = (len(filtered_df) + batch_size - 1) // batch_size

# Process in batches with progress bar
for i in tqdm(range(num_batches)):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(filtered_df))
    # Use .loc[] to avoid SettingWithCopyWarning
    filtered_df.loc[start_idx:end_idx-1, 'sentiment'] = filtered_df.loc[start_idx:end_idx-1, 'review/text'].apply(compute_sentiment)

# Print the first few rows of the DataFrame to verify the results
print(filtered_df.head())

# Remove the 'review/text' column from filtered_df
filtered_df.drop(columns=['review/text'], inplace=True)

# Print the final DataFrame to verify the result
print(filtered_df.head())

100%|█████████████████████████████████████████| 100/100 [08:22<00:00,  5.02s/it]

                      Title         User_id  review/score  \
1  Dr. Seuss: American Icon  A30TK6U7DNS82R           5.0   
3  Dr. Seuss: American Icon  A2MVUWT453QH61           4.0   
4  Dr. Seuss: American Icon  A22X4XUPKF66MR           4.0   
5  Dr. Seuss: American Icon  A2F6NONFUDB6UK           4.0   
6  Dr. Seuss: American Icon  A14OJS0VWMOSWO           5.0   

                                         review/text  sentiment  
1  I don't care much for Dr. Seuss but after read...     0.9876  
3  Theodore Seuss Geisel (1904-1991), aka &quot;D...     0.9807  
4  Philip Nel - Dr. Seuss: American IconThis is b...     0.9803  
5  "Dr. Seuss: American Icon" by Philip Nel is a ...     0.9939  
6  Theodor Seuss Giesel was best known as 'Dr. Se...     0.9658  
                      Title         User_id  review/score  sentiment
1  Dr. Seuss: American Icon  A30TK6U7DNS82R           5.0     0.9876
3  Dr. Seuss: American Icon  A2MVUWT453QH61           4.0     0.9807
4  Dr. Seuss: American Icon  A

In [50]:
# # Remove 'review/text' column from filtered_df
# filtered_df.drop(columns=['review/text'], inplace=True)


# Inspect sentiment scores
filtered_df.head()


,Title,User_id,review/score,sentiment
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,5.0,0.9876
3,Dr. Seuss: American Icon,A2MVUWT453QH61,4.0,0.9807
4,Dr. Seuss: American Icon,A22X4XUPKF66MR,4.0,0.9803
5,Dr. Seuss: American Icon,A2F6NONFUDB6UK,4.0,0.9939
6,Dr. Seuss: American Icon,A14OJS0VWMOSWO,5.0,0.9658


In [51]:
# Update 'score' column with the product of 'score' and 'sentiment'
filtered_df['score'] = filtered_df['review/score'] * filtered_df['sentiment']


In [128]:
filtered_df.head()

,Title,User_id,score
0,Dr. Seuss: American Icon,A30TK6U7DNS82R,4.9380
1,Dr. Seuss: American Icon,A2MVUWT453QH61,3.9228
2,Dr. Seuss: American Icon,A22X4XUPKF66MR,3.9212
3,Dr. Seuss: American Icon,A2F6NONFUDB6UK,3.9756
4,Dr. Seuss: American Icon,A14OJS0VWMOSWO,4.8290


In [53]:
filtered_df.drop(columns=['review/score', 'sentiment'], inplace=True)

In [54]:
filtered_df.head()

,Title,User_id,score
1,Dr. Seuss: American Icon,A30TK6U7DNS82R,4.9380
3,Dr. Seuss: American Icon,A2MVUWT453QH61,3.9228
4,Dr. Seuss: American Icon,A22X4XUPKF66MR,3.9212
5,Dr. Seuss: American Icon,A2F6NONFUDB6UK,3.9756
6,Dr. Seuss: American Icon,A14OJS0VWMOSWO,4.8290


In [126]:
# The limited user data did not present an opportunity to cluster the groups together.
# Instead, I used the review score and sentiment analysis together to determine which titles the user enjoyed,
# and recommended well rated books in the same categories and by the same authors.

# Create df of highly rated books
relevant_books = books_df[books_df['ratingsCount'] >= 4]

def recommend_books(User_id):
    # Step 1: Filter Reviews
    filtered_df.reset_index(drop=True, inplace=True)  # Reset index
    matching_titles = filtered_df.loc[(filtered_df['User_id'] == User_id) & (filtered_df['score'] >= 3.5), 'Title']
    print(matching_titles)
    # Add matching titles to pref_list
    pref_titles = []
    pref_titles.extend(matching_titles)
    
    recommended_books = []
    pref_authors= []
    pref_categories = []
    
    for title in pref_titles:
    # Look up title in relevant_books
        book_row = relevant_books[relevant_books['Title'] == title]
    
    if not book_row.empty:
        # Extract author and category number from the book_row
        author = book_row['author_encoded'].iloc[0]
        category = book_row['category_encoded'].iloc[0]
        
        # Add author to pref_authors and category number to pref_categories
        pref_authors.append(author)
        pref_categories.append(category)
    for author in pref_authors:
    # Search relevant_books for books with matching author
        matching_author_books = relevant_books[relevant_books['author_encoded'] == author]
    
    # Add matching titles to recommended_books
        recommended_books.extend(matching_author_books['Title'].tolist())

    for category in pref_categories:
        # Search relevant_books for books with matching category
        matching_category_books = relevant_books[relevant_books['category_encoded'] == category]

        # Add matching titles to recommended_books
        recommended_books.extend(matching_category_books['Title'].tolist())

    # Remove duplicates from recommended_books
    recommended_books = list(set(recommended_books))

    print("Recommended Books:")
    for book in recommended_books:
        print(book)
    


In [91]:
import random

# Get a random row from filtered_df
random_row = filtered_df.sample(n=1)

# Get the User_id from the random row
random_User_id = random_row['User_id'].iloc[0]

print("Random User_id:", random_User_id)

Random User_id: A1PILXSH4D277C


In [127]:
recommend_books('A1PILXSH4D277C')

34190                    Story Princess (Harlequin Romance)
41349     The Brazilian's Blackmailed Bride (Harlequin P...
41392                    Really Hot! (Harlequin Temptation)
46945           The Disobedient Virgin (Harlequin Presents)
70652     Convenient Bridegroom (Society Weddings) (Harl...
74850                                     Fatherhood Fever!
81549     Irresistible Prince (Royal Weddings) (Harlequi...
83482     Falling For A Father Of Four (Fabulous Fathers...
101495                                      Shattered Trust
129162                                       Naked in Death
152358                                   DENIM AND DIAMONDS
163370                   Her Body Of Work (Harlequin Blaze)
175975           Saving Grace (Harlequin Presents No. 1543)
219244                               Down from the Mountain
246361            One Night In Texas (Harlequin Temptation)
286501                                          LOVE'S DUEL
316238                                  